In [2]:
import pandas as pd
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import re

In [3]:
df = pd.read_csv('/Users/com/Documents/GitHub/advanced_project/jiwon_project/csv_files/NY_Airbnb_original_df.csv')

In [4]:
# 전처리 1 - id int로 변경
df['id'] = df['id'].astype(int)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22308 entries, 0 to 22307
Data columns (total 72 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            22308 non-null  int64  
 1   source                                        22308 non-null  object 
 2   name                                          22308 non-null  object 
 3   description                                   21903 non-null  object 
 4   neighborhood_overview                         12262 non-null  object 
 5   host_id                                       22308 non-null  int64  
 6   host_name                                     22288 non-null  object 
 7   host_since                                    22288 non-null  object 
 8   host_location                                 17025 non-null  object 
 9   host_about                                    13391 non-null 

In [6]:
# booking_info
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float) 
df['instant_bookable'] =  df['instant_bookable'].map({'f':0, 't':1}) #instant_bookable = 호스트의 허락없이 에어비엔비로 거래 가능한 
df['is_long_term'] = (df['minimum_nights'] >= 28).astype(int) #오래 머무른 사람 28일 이상 기준 

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12444\965363162.py:2: SyntaxWarning: invalid escape sequence '\$'
  df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)


In [ ]:
# amenities_info
def parse_amenities(x):
    try:
        return [a.strip().strip('"').strip("'") for a in ast.literal_eval(x)]
    except:
        return []
    
df['amenities'] = df['amenities'].apply(parse_amenities) #편의시설 유무 
df['amenities_cnt'] = df['amenities'].apply(len)


In [ ]:
# host_info
# neighborhood_overview 결측치 많아서 유무대체 
df['neighborhood_overview_exists'] = df['neighborhood_overview'].notnull().astype(int) #동네 개요 = 동네의 특성 의미 , neighborhood_overview 사람들이 살고 교류하는 지리적으로 정의된 지역


In [ ]:
# name 글자수기준 중앙값으로 그룹
df['name_length'] = df['name'].fillna('').astype(str).apply(len) 
med_length = df['name_length'].median()

def name_length_group(length, med):
    if length == 0:
        return 'empty'
    elif length > med:
        return 'long'
    else:
        return 'short_or_med'
    
df['name_length_group'] = df['name_length'].apply(lambda x: name_length_group(x, med_length))

In [15]:
df.isnull().sum()

id                                  0
source                              0
name                                0
description                       405
neighborhood_overview           10046
                                ...  
is_long_term                        0
amenities_cnt                       0
neighborhood_overview_exists        0
name_length                         0
name_length_group                   0
Length: 77, dtype: int64

In [ ]:
# description 글자수기준(결측치 405) 평균으로 그룹
df['description_length'] = df['description'].fillna('').astype(str).apply(len) #설명문 = 소개글 
avg_length = df['description_length'].mean()

def name_length_group(length, avg):
    if length == 0:
        return 'empty'
    elif length > avg:
        return 'long'
    else:
        return 'short_or_avg'
    
df['description_length_group'] = df['description_length'].apply(lambda x: name_length_group(x, avg_length))

In [17]:
# host_about (결측치8917) 평균(243) 중앙값(81) 중앙값기준으로 그룹
df['host_about_length'] = df['host_about'].fillna('').astype(str).apply(len)
med_length = df['host_about_length'].median()

def name_length_group(length, med):
    if length == 0:
        return 'empty'
    elif length > med:
        return 'long'
    else:
        return 'short_or_med'
df['host_about_length_group'] = df['host_about_length'].apply(lambda x: name_length_group(x, med_length))

In [ ]:
#host_identity_verified/host_has_profile_pic /host_is_superhost  
# True / Flase 1과 0으로 대체 (결측치 20/20/350 0으로 대체함)
df['host_identity_verified']=df['host_identity_verified'].fillna('f').map({'t': 1, 'f': 0}).astype(int) #호스트 신원확인

df['host_has_profile_pic']=df['host_has_profile_pic'].fillna('f').map({'t': 1, 'f': 0}).astype(int)

df['host_is_superhost']=df['host_is_superhost'].fillna('f').map({'t': 1, 'f': 0}).astype(int)


In [19]:
# host_response_time 결측치는 중앙값으로 치환후 점수
response_time_score_map = { 
    'within an hour': 4,
    'within a few hours': 3,
    'within a day': 2,
    'a few days or more': 1
}
df['host_response_time_score'] = df['host_response_time'].map(response_time_score_map)

# 2. response_time_score 컬럼의 중앙값 계산
med_score_for_fillna = df['host_response_time_score'].median()

# 3. response_time_score 컬럼의 NaN을 계산된 중앙값으로 대체 
df['host_response_time_score'] = df['host_response_time_score'].fillna(med_score_for_fillna)

# host_response_time 칼럼에는 여전히 nan값 존재함
# response_time_score 칼럼만 중앙값대체 
